In [1]:
import utils
import torch
from tqdm import tqdm

/home/venom/intel/oneapi/intelpython/latest/envs/pytorch-gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/venom/intel/oneapi/intelpython/latest/envs/pytorch-gpu/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/venom/intel/oneapi/intelpython/python3.9/envs/pytorch-gpu/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
model = utils.give_model()
model.load_state_dict(torch.load("./model/model.pt", map_location="cpu"))
train_dataloader, valid_dataloader, test_dataloader = utils.give_data_loaders()
batch = next(iter(test_dataloader))
criterion = utils.give_criterion()

## Quantize

In [3]:
model.eval()
model.cpu()
print()

In [4]:
def validate(model, val_loader, criterion):
    total_loss = 0.0

    with torch.no_grad(), tqdm(total=len(val_loader), desc="Validation") as progress_bar:
        for batch in val_loader:
            images = batch["image"]
            masks = batch["mask"]

            logits_mask = model(images)
            loss = criterion(logits_mask, masks)

            total_loss += loss.item()
            progress_bar.update(1)

    return total_loss / len(val_loader)


In [5]:
input_data = [((torch.randn(1, 3, 256, 256), ), torch.tensor([0])) for _ in range(100)]


In [8]:
from speedster import optimize_model

# Run Speedster optimization
optimized_model = optimize_model(
    model,
    input_data=input_data,
    device='cpu',
    store_latencies=True,
    optimization_time="unconstrained"
    
)

2023-07-12 20:37:06 | INFO     | Running Speedster on CPU
2023-07-12 20:37:07 | WARNING  | Missing Frameworks: tensorflow.
 Please install them to include them in the optimization pipeline.
2023-07-12 20:37:07 | WARNING  | Missing Compilers: onnxruntime, deepsparse.
 Please install them to include them in the optimization pipeline.
2023-07-12 20:37:12 | INFO     | Benchmark performance of original model
2023-07-12 20:37:18 | INFO     | Original model latency: 0.042703731060028075 sec/iter
2023-07-12 20:37:18 | INFO     | [1/2] Running PyTorch Optimization Pipeline
2023-07-12 20:37:18 | INFO     | Optimizing with TorchScriptCompiler and q_type: None.
2023-07-12 20:37:19 | WARNING  | Unable to trace model with torch.fx
2023-07-12 20:37:21 | INFO     | Optimized model latency: 0.03694915771484375 sec/iter
2023-07-12 20:37:21 | INFO     | [2/2] Running ONNX Optimization Pipeline
2023-07-12 20:37:21 | INFO     | Optimizing with OpenVINOCompiler and q_type: None.
[ INFO ] The model was conve